In [1]:
import pandas as pd
import numpy as np
import os
import glob
import pathlib
import re
import janome
import jaconv

In [2]:
p_dic = pathlib.Path('dic')

for i in p_dic.glob('*.txt'):
    with open (i, 'r', encoding='utf-8') as f:
        x = [i.replace('\n', '').split(':') for i in f.readlines()]
        
posi_nega_df = pd.DataFrame(x, columns = ['基本形', '読み', '品詞', 'スコア'])
posi_nega_df['読み'] = posi_nega_df['読み'].apply(lambda x : jaconv.hira2kata(x))
posi_nega_df = posi_nega_df[~posi_nega_df[['基本形', '読み', '品詞']].duplicated()]

In [3]:
posi_nega_df.head()

,基本形,読み,品詞,スコア
0,優れる,スグレル,動詞,1
1,良い,ヨイ,形容詞,0.999995
2,喜ぶ,ヨロコブ,動詞,0.999979
3,褒める,ホメル,動詞,0.999979
4,めでたい,メデタイ,形容詞,0.999645


In [4]:
posi_nega_df.shape

(55112, 4)

In [5]:
p_temp = pathlib.Path('text')
article_list = []
for p in p_temp.glob('**/*.txt'):
    if len(str(p).split('/')) >2:
        media = str(p).split('/')[1]
        file_name = str(p).split('/')[2]
    
        if file_name != 'LICENSE.txt':
            with open(p, 'r') as f:
                article = f.readlines()
                article = [re.sub(r'[\n \u3000]', '', i) for i in article]
            article_list.append([media, article[0], article[1], article[2], ''.join(article[3:])])

        else:
            continue
article_df = pd.DataFrame(article_list)
article_df.head()

,0,1,2,3,4
0,dokujo-tsushin,http://news.livedoor.com/article/detail/4778030/,2010-05-22T14:30:00+0900,友人代表のスピーチ、独女はどうこなしている？,もうすぐジューン・ブライドと呼ばれる６月。独女の中には自分の式はまだなのに呼ばれてばかり……...
1,dokujo-tsushin,http://news.livedoor.com/article/detail/4778031/,2010-05-21T14:30:00+0900,ネットで断ち切れない元カレとの縁,携帯電話が普及する以前、恋人への連絡ツールは一般電話が普通だった。恋人と別れたら、手帳に書か...
2,dokujo-tsushin,http://news.livedoor.com/article/detail/4782522/,2010-05-23T11:00:00+0900,相次ぐ芸能人の“すっぴん”披露その時、独女の心境は？,「男性はやっぱり、女性の“すっぴん”が大好きなんですかね」と不満そうに話すのは、出版関係で働...
3,dokujo-tsushin,http://news.livedoor.com/article/detail/4788357/,2010-05-25T14:00:00+0900,ムダな抵抗！？加齢の現実,ヒップの加齢による変化は「たわむ→下がる→内に流れる」、バストは「そげる→たわむ→外に流れる...
4,dokujo-tsushin,http://news.livedoor.com/article/detail/4788362/,2010-05-26T14:00:00+0900,税金を払うのは私たちなんですけど！,6月から支給される子ども手当だが、当初は子ども一人当たり月額2万6000円が支給されるはずだ...


In [7]:
news_df = article_df[article_df[0] == 'sports-watch'].reset_index(drop = True)

from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *

t = Tokenizer()
char_filters = [UnicodeNormalizeCharFilter()]
analyzer = Analyzer(char_filters, t)

# word_lists = []
# for i, row in news_df.iterrows():
#     for t in analyzer.analyze(row[4]):
#         print(t)

TypeError: __init__() takes 1 positional argument but 3 were given